In [1]:
%matplotlib inline
import keras
from keras.models import Sequential
from keras.utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
from keras.datasets import cifar10
from keras import regularizers, optimizers
import numpy as np

Using TensorFlow backend.


In [2]:
NUM_CLASSES = 10

In [3]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

y_train = to_categorical(y_train, NUM_CLASSES)
y_test = to_categorical(y_test, NUM_CLASSES)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

In [4]:
baseMapNum = 32
weight_decay = 1e-4

In [5]:
model = Sequential()
model.add(Conv2D(baseMapNum, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay), input_shape=x_train.shape[1:]))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Conv2D(baseMapNum, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

model.add(Conv2D(2*baseMapNum, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Conv2D(2*baseMapNum, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.3))

model.add(Conv2D(4*baseMapNum, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Conv2D(4*baseMapNum, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Flatten())
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(NUM_CLASSES,activation='softmax'))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
activation_1 (Activation)    (None, 32, 32, 32)        0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 32, 32, 32)        128       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 32, 32)        9248      
_________________________________________________________________
activation_2 (Activation)    (None, 32, 32, 32)        0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 32, 32, 32)        128       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 16, 16, 32)        0         
__________

In [6]:
# opt = keras.optimizers.rmsprop(lr=0.0001, decay=1e-6)
model.compile(loss='categorical_crossentropy',
        optimizer='adam',
        metrics=['accuracy'])

In [7]:
DATA_AUG = True
BATCH_SIZE = 64
NUM_EPOCHS = 100

In [8]:
if DATA_AUG :
    print("-------------Using Data augmentation------------")
     # This will do preprocessing and realtime data augmentation:
    datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False)  # randomly flip images
    
    datagen.fit(x_train)
    model.fit_generator(datagen.flow(x_train, y_train, batch_size=BATCH_SIZE),
                        steps_per_epoch=x_train.shape[0] // BATCH_SIZE,
                        epochs=NUM_EPOCHS,verbose=1,validation_data=(x_test,y_test))
    
else :
    print("-----Not Using Data augmentation---------------")
    model.fit(x_train, y_train,
              batch_size=BATCH_SIZE*4,
              epochs=epochs,
              validation_data=(x_test, y_test),
              shuffle=False,
              callbacks=[EarlyStopping(min_delta=0.001, patience=3)])

-------------Using Data augmentation------------
Epoch 1/100
781/781 [==============================] - 31s 40ms/step - loss: 1.7479 - acc: 0.4143 - val_loss: 1.4372 - val_acc: 0.4954
Epoch 2/100
781/781 [==============================] - 28s 36ms/step - loss: 1.3133 - acc: 0.5477 - val_loss: 1.3151 - val_acc: 0.5546
Epoch 3/100
781/781 [==============================] - 27s 35ms/step - loss: 1.1579 - acc: 0.6102 - val_loss: 1.0252 - val_acc: 0.6697
Epoch 4/100
781/781 [==============================] - 27s 35ms/step - loss: 1.0503 - acc: 0.6566 - val_loss: 0.9684 - val_acc: 0.6879
Epoch 5/100
781/781 [==============================] - 27s 35ms/step - loss: 0.9831 - acc: 0.6834 - val_loss: 0.9419 - val_acc: 0.7025
Epoch 6/100
781/781 [==============================] - 28s 36ms/step - loss: 0.9337 - acc: 0.7067 - val_loss: 0.8631 - val_acc: 0.7342
Epoch 7/100
781/781 [==============================] - 27s 35ms/step - loss: 0.9022 - acc: 0.7209 - val_loss: 0.9252 - val_acc: 0.7086
Epoch 

Epoch 61/100
781/781 [==============================] - 29s 38ms/step - loss: 0.5728 - acc: 0.8611 - val_loss: 0.5760 - val_acc: 0.8691
Epoch 62/100
781/781 [==============================] - 29s 37ms/step - loss: 0.5758 - acc: 0.8629 - val_loss: 0.5562 - val_acc: 0.8730
Epoch 63/100
781/781 [==============================] - 27s 35ms/step - loss: 0.5697 - acc: 0.8616 - val_loss: 0.5881 - val_acc: 0.8601
Epoch 64/100
781/781 [==============================] - 27s 35ms/step - loss: 0.5679 - acc: 0.8645 - val_loss: 0.5776 - val_acc: 0.8701
Epoch 65/100
781/781 [==============================] - 28s 35ms/step - loss: 0.5701 - acc: 0.8639 - val_loss: 0.5785 - val_acc: 0.8685
Epoch 66/100
781/781 [==============================] - 28s 36ms/step - loss: 0.5715 - acc: 0.8640 - val_loss: 0.5455 - val_acc: 0.8768
Epoch 67/100
781/781 [==============================] - 28s 36ms/step - loss: 0.5725 - acc: 0.8637 - val_loss: 0.5694 - val_acc: 0.8703
Epoch 68/100
781/781 [==========================

In [9]:
model.save("cifar4_CNN_Epoch_"+ str(NUM_EPOCHS) +"_Daug_"+ str(DATA_AUG) +".h5")